The notebook demonstrates how to solve the motions of a double pendulum. <br>

The tutorial contains functions of four first-order ODEs that build up the system. The system is solved by the Euler, Midpoint, Runge-Kutta methods we developed previously. 

In [1]:
%pip install -q ipywidgets matplotlib numpy scipy pandas
import numpy  as np
import pandas as pd
import ode    as odeLib        # our ODE routines
import matplotlib.pyplot as plt
from   math                 import radians, sin, cos, pi
from   scipy                import integrate
from   ipywidgets           import *
from   IPython.display      import display

Note: you may need to restart the kernel to use updated packages.


In [2]:
#
# define the problem
#
class parameters:                 # parameters for functions, default values
    l1 = 1.
    l2 = 1.
    m1 = 2.
    m2 = 2.
    g  = 9.8
# 
# define functions for ODE solvres, need to be in f(t,x,par) format
#
# damping harmonic oscillator, dx/dt for dotx and x.

ndim   = 4
vlabel = ["the1", "the2", "w1", "w2"]            # labels for the variables
# All the angles are in radians.
# the1: the angular position of mass 1. 
# the2: the angular position of mass 2.
# w1  : the angular velocity of mass 1.
# w2  : the angular velocity of mass 2.

def f0(t,x,par): # dthe1/dt = w1
    return x[2]
def f1(t,x,par): # dthe2/dt = w2
    return x[3]
def f2(t,x,par): # dw1/dt 
    g  = par.g
    m1 = par.m1
    m2 = par.m2
    l1 = par.l1
    l2 = par.l2
    
    numerator   = -g*(2*m1+m2)*sin(x[0])-m2*g*sin(x[0]-2*x[1]) - 2*sin(x[0]-x[1])*m2*(x[3]*x[3]*l2 + x[2]*x[2]*l1*cos(x[0]-x[1]))
    denominator = l1*(2*m1+m2-m2*cos(2*x[0]-2*x[1]))
    return numerator/denominator

def f3(t,x,par): # dw2/dt 
    g  = par.g
    m1 = par.m1
    m2 = par.m2
    l1 = par.l1
    l2 = par.l2
    
    numerator   = 2*sin(x[0]-x[1])*(x[2]*x[2]*l1*(m1+m2) + g*(m1+m2)*cos(x[0]) + x[3]*x[3]*l2*m2*cos(x[0]-x[1]))
    denominator = l2*(2*m1 + m2 - m2*cos(2*x[0] - 2*x[1]))
    
    return numerator/denominator

def xy(theta1, theta2, par):
    # given the time series of theta1 and theta2 and l1 and l2, 
    # return locations of the two masses as a function of time.
    m1 = par.m1
    m2 = par.m2
    l1 = par.l1
    l2 = par.l2
    
    x1 = l1*np.sin(theta1)
    y1 = -l1*np.cos(theta1)
    x2 = x1 + l2*np.sin(theta2)
    y2 = y1 - l2*np.cos(theta2)
    
    return x1,y1,x2,y2

def ode_solver(time, xInit, method, ndim, par):
    # method supports three options
    # - 'eular'
    # - 'midpoint'
    # - 'runge_kutta'
    
    nstep = len(time)
    xout  = np.empty((nstep,ndim))
    
    hs    = time[1] - time[0]
    xtmp  = xInit
    f     = [f0, f1, f2, f3]
    for i, t in enumerate(time):
        xout[i][:] = xtmp
        if method  == 'euler':
            xtmp   = odeLib.euler(t,xtmp,f,ndim,hs,par)
        elif method == 'midpoint':
            xtmp   = odeLib.midpoint(t,xtmp,f,ndim,hs,par)
        elif method == 'runge_kutta':
            xtmp   = odeLib.runge_kutta(t,xtmp,f,ndim,hs,par)

    return xout

In [3]:
def main(l1,l2,m1,m2, xi1,xi2,xi3,xi4):
    
    Nt     = 1000
    t      = np.linspace(0., 10., Nt)

    par        = parameters()
    par.l1     = l1
    par.l2     = l2
    par.m1     = m1
    par.m2     = m2
    
    xInit = [radians(xi1), radians(xi2), radians(xi3), radians(xi4)]
    
    xi     = xInit 
    x1a    = ode_solver(t, xi, 'euler',       ndim, par)

    x1,y1,x2,y2 = xy(x1a[:,0],x1a[:,1], par)
    
    
    fig    = plt.figure()
    
    plt.plot(t, x1a[:,0]/pi*180, 'g--',  label='theta1',    linewidth=2.5)
    plt.plot(t, x1a[:,1]/pi*180, 'b:',   label='theta2', linewidth=2)
    
    plt.grid()
    plt.ylim(-180., 180.)
    plt.xlabel("Time, $t$")
    plt.ylabel("Amplitude, $a$")
    plt.legend()
    
    fig1   = plt.figure()
    plt.plot(x1a[:,0]/pi*180, x1a[:,1]/pi*180, 'g--',    linewidth=2.5)
    plt.xlabel(r"$\Theta_1$ [deg]")
    plt.ylabel(r"$\Theta_2$ [deg]")    
    
    fig2   = plt.figure()
    plt.plot(x1, y1, 'g--', label='ball1',   linewidth=2.5)
    plt.plot(x2, y2, 'k--', label='ball2',   linewidth=2.5)
    plt.xlabel("x")
    plt.ylabel("y")
    
    return fig, fig1, fig2

In [4]:
w = interactive(main, l1 = (0.5, 10, 0.5), 
                      l2 = (0.5, 10, 0.5), 
                      m1 = (0.5, 10, 0.5), 
                      m2 = (0.5, 10, 0.5),  
                      xi1= fixed(30), # initial angular positions in degrees.
                      xi2= fixed(30),
                      xi3= fixed(0),
                      xi4= fixed(0))
display(w)

interactive(children=(FloatSlider(value=5.0, description='l1', max=10.0, min=0.5, step=0.5), FloatSlider(value…